In [1]:
!pip install torch torchvision
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as ds
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt

     ------------------------------------- 172.3/172.3 MB 13.1 MB/s eta 0:00:00
     ---------------------------------------- 1.2/1.2 MB 25.2 MB/s eta 0:00:00


In [ ]:
transform_train = transforms.Compose([
        transforms.Resize(256),
        transforms.RandomCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))])

transform_test = transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))])

batch_size = 2

cifar10_train =  ds.CIFAR10('../', train = True, transform = transform_train, target_transform = None, download = True)
cifar10_test = ds.CIFAR10('../', train = False, transform = transform_test, target_transform = None, download = True)


train_loader = DataLoader(cifar10_train, batch_size = batch_size, shuffle = True, num_workers = 2, drop_last = True)

test_loader = DataLoader(cifar10_test, batch_size = batch_size, shuffle = False, num_workers = 2, drop_last = True)

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.layer = nn.Sequential(
            # Conv1, MaxPool1, Conv2, MaxPool2, Conv3, MaxPool3 구현            
        nn.Conv2d(in_channels = 3, out_channels = 96, kernel_size = 7, stride = 4, padding = 3), #[96, 56, 56]
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1 ), #[96 * 28 * 28]

        nn.Conv2d(in_channels = 96, out_channels = 256, kernel_size = 5, stride = 1, padding = 2),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1), # [256, 14, 14]

        nn.Conv2d(in_channels = 256, out_channels = 384, kernel_size = 3, stride = 1, padding = 0),  ###여기서 kernel_size가 3인데, 스트라이드를 1 주면 데이터 손실.
                                                                                                        # 패딩값이 0이기 때문에, [256,14,14] -> [384, 12, 12]로 됨.
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 7, stride = 1, padding = 0), #[384, 12, 12]



        
        )
        self.fc_layer = nn.Sequential(                              
            # Linear1, Linear2 구현
            nn.Linear(384 * 12 * 12, 4096),
            nn.ReLU(),

            nn.Linear(4096, 10)

        )       
        
    def forward(self,x):
        out = self.layer(x)
        out = out.view(batch_size,-1)
        out = self.fc_layer(out)
        return out

In [ ]:
learning_rate = 0.001
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
num_epoch = 2
for i in range(num_epoch):
    for j,[image,label] in enumerate(train_loader):
        x = image.to(device)
        y_= label.to(device)
        
        optimizer.zero_grad()
        output = model.forward(x)
        loss = loss_func(output, y_)
        loss.backward()
        optimizer.step()
        
    print(loss)

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for image, label in test_loader:
        
        x = image.to(device)
        y_= label.to(device)

        output = model.forward()
        
        _,output_index = torch.max(output,1)
        
        total += label.size(0)
        
        correct += (output_index == y_).sum.float()
    
    print("Accuracy of Test Data: {}%".format(100*correct / total))